In [1]:
import pandas as pd
import numpy as np 
from datetime import date
import matplotlib.pyplot as plot
import numpy as np
import random
import operator
from functools import reduce
from sklearn.preprocessing import MinMaxScaler
import torch

In [4]:
import matplotlib.pyplot as plt

import pandas as pd

import numpy as np
from datetime import date
import math

In [86]:
#生成居民用水行为，1：away;2:sleep;3:bath;4:showering;5:others
user1=[2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,4,4,4,4,1,1,1,1,5,5,5,5,5,5,5,5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,5,5,5,5,3,3,3,3,5,5,5,5,3,3,3,3,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
user2=[2,2,2,2,5,5,5,5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,5,5,5,5,5,5,5,5,4,4,4,4,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
user3=[5,5,5,5,5,5,5,5,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,4,4,4,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,5,5,5,5,5,5,5,5,4,4,4,4,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
user4=[2,2,2,2,2,2,2,2,2,2,2,2,5,4,4,4,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,3,3,3,5,5,5,5,5,5,5,5,4,4,4,4,5,5,5,5,2,2,2,2,2,2,2,2]
user5=[2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,4,4,4,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,2,2,2,2,2,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]

In [87]:
userpattern0521200=[]
for j in range(200):
    pattern=[]
    for i in range(365):
        x=random.randint(1,5)
        if x==1:
            pattern.append(user1)
        if x==2:
            pattern.append(user2)
        if x==3:
            pattern.append(user3)
        if x==4:
            pattern.append(user4)
        if x==5:
            pattern.append(user5)
    pattern=np.array(pattern).reshape(1,365*96)
    userpattern0521200.append(pattern)
userpattern0521200=np.array(userpattern0521200)


In [88]:
userpattern0521200=userpattern0521200.reshape(200,-1)

In [5]:
pattern = np.load('G:/科研/代码/July_coding/code/data/0519 2000userpattern.npy')

In [6]:
pattern200 = pattern[:200,1:]
pattern200.shape

(200, 23328)

In [7]:
def datagenerate(next_list,userpattern):
    Temp=[]
    Pbase=[]
    Power=np.zeros(len(next_list),dtype=int)
    for j in range(200):
        state_norm=np.zeros(len(next_list),dtype=int)
        T_list_1=np.zeros(len(next_list),dtype=int)
        T_list_1[0]=80
        state_norm[0]=0
        pattern=userpattern[j]
        for i in range(len(next_list)-1):
            if state_norm[i] == 1 and (pattern[i] == 1 or pattern[i] == 2 or pattern[i] == 5):
                T_list_1[i + 1] = T_list_1[i] + 6.4
                if T_list_1[i + 1] >= 100 or next_list[i + 1] == -1:
                    state_norm[i + 1] = 0
                else:
                    state_norm[i + 1] = 1
            elif state_norm[i] == 1 and pattern[i] == 4:
                T_list_1[i + 1] = T_list_1[i] - 8.6
                if next_list[i + 1] == -1:
                    state_norm[i + 1] = 0
                else:
                    state_norm[i + 1] = 1
            elif state_norm[i] == 1 and pattern[i] == 3:
                T_list_1[i + 1] = T_list_1[i] - 4.6
                if next_list[i + 1] == -1:
                    state_norm[i + 1] = 0
                else:
                    state_norm[i + 1] = 1
            elif state_norm[i] == 0 and (pattern[i] == 1 or pattern[i] == 2 or pattern[i] == 5):
                T_list_1[i + 1] = T_list_1[i] - 1.142
                if T_list_1[i + 1] <= 60 or next_list[i + 1] == 1:
                    state_norm[i + 1] = 1
                else:
                    state_norm[i + 1] = 0
            elif state_norm[i] == 0 and pattern[i] == 4:
                T_list_1[i + 1] = T_list_1[i] - 15.143
                if T_list_1[i + 1] <= 60 or next_list[i + 1] == 1:
                    state_norm[i + 1] = 1
                else:
                    state_norm[i + 1] = 0
            else:
                T_list_1[i + 1] = T_list_1[i] - 8.14
                if T_list_1[i + 1] <= 60 or next_list[i + 1] == 1:
                    state_norm[i + 1] = 1
                else:
                    state_norm[i + 1] = 0
        p_norm = state_norm.copy()
        p_norm = np.where(state_norm == 1, 3, 0)
        Power=Power+p_norm
        Pbase.append(p_norm)
        Temp.append(T_list_1)
    Temp=np.array(Temp)
    Pbase = np.array(Pbase)
    return Power,Temp,Pbase

In [8]:
ds = np.zeros(96,dtype=int).repeat(243)
ZP,ZT,ZP1 = datagenerate(ds, pattern200)

In [9]:
ZP1[125,96*1:96*2]

array([3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 0, 0, 0])

In [10]:
#生成up和down不同的96组信号
down = []
for i in range(96):
    row = []
    for j in range(96):
        if j < i:
            row.append(0)
        else:
            row.append(-1)
    down.append(row)
down = np.array(down)

up = []
for i in range(96):
    row = []
    for j in range(96):
        if j < i:
            row.append(0)
        else:
            row.append(1)
    up.append(row)
up = np.array(up)

# 生成一天的灵活度
def flexoneday(p_base, t_base, dr_sig, userpattern):
    p_flex = np.zeros(96 , dtype=int)
    for i in range(200):
        flex = np.zeros(96, dtype=int).tolist()
        temp = t_base[i]
        p = p_base[i]
        pattern = userpattern[i]
        for j in range(96 - 1):
            if dr_sig[j] == 0:
                flex[j] = 0
            elif dr_sig[j] == -1:
                flex[j] = p[j]
                p[j] = 0
                if pattern[j] == 1 or pattern[j] == 2 or pattern[j] == 5:
                    temp[j + 1] = temp[j] - 1.142
                elif pattern[j] == 3:
                    temp[j + 1] = temp[j] - 8.14
                else:
                    temp[j + 1] = temp[j] - 15.143
                if temp[j] <= 60:
                    flex[j] = 0  # 温控信号与dr信号对冲，此后灵活度均降为零
            else:  # 正1信号代表此刻功率变为最大功率
                flex[j] = 3 - p[j]
                p[j] = 3
                if pattern[j] == 1 or pattern[j] == 2 or pattern[j] == 5:
                    temp[j + 1] = temp[j] + 6.4
                elif pattern[j] == 3:
                    temp[j + 1] = temp[j] - 4.6
                else:
                    temp[j + 1] = temp[j] - 8.6
                if temp[j] >= 100:
                    flex[j] = 0  # 温控信号与dr信号对冲，灵活度降为零
        p_flex = p_flex + np.array(flex)
    return p_flex


In [17]:
device = 'cpu'


In [19]:
ZP2 = torch.Tensor(ZP1).to(device)
ZP_ = torch.Tensor(ZP).to(device)
ZT_ = torch.Tensor(ZT).to(device)
pattern_ = torch.Tensor(pattern200).to(device)

ext_pbase = []
ext_ds = []
ext_pflex = []
p = ZP2[:, 96: 2* 96].clone()
t = ZT_[:, 96: 2* 96].clone()
pattern = pattern_[:, 96: 2* 96]
a = up[3].tolist()
flex = flexoneday(p, t, a, pattern).tolist()
ext_pflex += flex
ext_pbase +=ZP_[96: 2* 96].tolist()
ext_ds += a






In [106]:
ext_pbase = []
ext_ds = []
ext_pflex = []

p = ZP2[:, 96: 2* 96].copy()
t = ZT_[:, 96: 2* 96].copy()
pattern = pattern200[:, 96: 2* 96]
a = up[3].tolist()
flex = flexoneday(p, t, a, pattern).tolist()
ext_pflex += flex
ext_pbase +=ZP[96: 2* 96].tolist()
ext_ds += a

p = ZP1[:, 96: 2* 96].copy()
t = ZT[:, 96: 2* 96].copy()
a = down[50].tolist()
pattern = pattern200[:, 96: 2* 96]
flex = flexoneday(p, t, a, pattern).tolist()
ext_pflex += flex
ext_pbase += ZP[96: 2* 96].tolist()



In [85]:
ZP.shape

(23328,)

In [ ]:
ds_0 = np.zeros(96, dtype=int).tolist()
ext_pbase = []
ext_ds = []
ext_pflex = []
ext_tariff = []
for i in range(242):
    for j in range(96):
        p = ZP1[:, i*96:(i + 1)*96].copy()
        t = ZT[:, i*96:(i + 1)*96].copy()
        pattern = pattern200[:, i*96:(i + 1)*96]
        a = up[j].tolist()
        flex = flexoneday(p, t, a, pattern).tolist()
        ext_pflex += flex
        ext_pbase += ZP[i*96:(i + 1)*96].tolist()
        ext_ds += a
    for j in range(96):
        p = ZP1[:, i*96:(i + 1)*96].copy()
        t = ZT[:, i*96:(i + 1)*96].copy()
        a = down[j].tolist()
        pattern = pattern200[:, i*96:(i + 1)*96]
        flex = flexoneday(p, t, a, pattern).tolist()
        ext_pflex += flex
        ext_pbase += ZP[i*96:(i + 1)*96].tolist()
        ext_ds += a
    for j in range(14):
        p = ZP1[:, i*96:(i + 1)*96].copy()
        t = ZT[:, i*96:(i + 1)*96].copy()
        pattern = pattern200[:, i*96:(i + 1)*96]
        flex = flexoneday(p, t, ds_0, pattern).tolist()
        ext_pflex += flex
        ext_pbase += ZP[i*96:(i + 1)*96].tolist()
        ext_ds += ds_0
ext_pbase = np.array(ext_pbase)
ext_ds = np.array(ext_ds)
ext_pflex = np.array(ext_pflex)


In [89]:
ext_pflex = ext_pflex.reshape(-1)

In [90]:
data={'p_base':ext_pbase,'ds':ext_ds,'ext_pflex':ext_pflex}
df = pd.DataFrame(data)
df.to_csv('0804 dataset.csv')

In [75]:
ext_pflex1 = ext_pflex.reshape(-1,96,1)
ext_pflex1.shape

(49852, 96, 1)

In [78]:
ext_pflex2 = ext_pflex1[:-30*206,:,]
ext_pflex2.shape

(43672, 96, 1)

In [83]:
43672/206

212.0